In [538]:
import pandas as pd



# pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_colwidth')

data1 = pd.read_csv('../data/appartements1.csv')
data2 = pd.read_csv('../data/appartements2.csv')
appartements=pd.concat([data1,data2],axis=0,ignore_index=True)
df =appartements.copy()



In [539]:

##-pre-processing and preparation of the data 
from sklearn.model_selection import train_test_split
from preprocessing import Preprocessing 
p =Preprocessing()
X,y,vectorize =p.processing(df)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=0)


c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['etat'] =df['etat'].str.lower().str.replace(' ','')
c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['autre'] =df['autre'].str.lower()
c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

### Optimisation 

In [540]:
import pickle

file =open('../Models/Stacking.pkl','rb')
Stacking =pickle.load(file)



In [541]:
## Reformation of the dataframe 

df=pd.concat([X_train,X_test],axis=0)
y=pd.concat([y_train,y_test],axis=0)

df =pd.concat([df,y],axis=1)


In [542]:
import pandas as pd
data_dict ={'surface(m²)':120,
'pieces':3,
'chambres':2,
'salles de bains':3,
'etat':'bonétat',
'etage':'3etage',
'ans':'10to20',
'ville':'oujda',
'autre' :'parquet tv'
}
prediction =pd.DataFrame([data_dict])

In [543]:
prediction

city=prediction['ville']

In [544]:
import numpy as np

## this function prepare the value to be predicted 
def prepare_data_for_prediction(df):
    numerica_cols =df.select_dtypes(include=np.number)

    df['Description'] = (df['etat'].fillna('') +" " +df['etage'].fillna('') +" "+
                    " "+ df['ans'].fillna('') +" "+ df['ville'].fillna('') +" "+
                        df['autre'].fillna('')).str.replace('  ',' ').str.replace('nan','').str.lower().str.strip()
    v=df['Description']
    pd.set_option('display.max_columns', None)
    
    ##--apply TF-IDF on the appartment we want to predict the price 
    X =vectorize.transform(v)
    X_tfidf_pred =pd.DataFrame(X.toarray(),columns=vectorize.get_feature_names_out())
    numeric_cols_pred =prediction[list(numerica_cols)]
    to_predict =pd.concat([numeric_cols_pred,X_tfidf_pred],axis=1)

    return to_predict


In [545]:
len(df.columns)

to_predict =prepare_data_for_prediction(prediction)

len(to_predict.columns)

246

In [546]:
from sklearn.metrics.pairwise import cosine_similarity

###---calculate the prediction 
to_predict =prepare_data_for_prediction(prediction)
price =Stacking.predict(to_predict)

#--- Add predicted price 
to_predict['prix(DHs)']=price


print(f"the price of this apartment will be almost: {price[0]} DHs ")

the price of this apartment will be almost: 917576.6653029998 DHs 


In [547]:

##-------calculate the similarity by cosine metric
df = df[df[city.values[0]]!= 0]
similarities = cosine_similarity(df, to_predict)
df['similarity'] = similarities

def similar_appartement(index):
    
    similare_att =appartements[appartements.index==index]
    return similare_att

In [548]:

i =1

for index in df['similarity'].sort_values(ascending=False).index[:2000]:
    similare_att = similar_appartement(index)
    predected_price = to_predict['prix(DHs)'].values[0]
    recommanded_price = float(similare_att['prix(DHs)'].values[0].replace(' DH','').replace(' EUR',''))

    price_difference = (abs(predected_price - recommanded_price))<150000

        
    if price_difference:
        print(f"you can visit recommended apartment from this link:\n{similare_att['lienArticle'].iloc[0]}")
        print("====================================================")
        break


you can visit recommended apartment from this link:
https://www.mubawab.ma/fr/a/7779889/appartement-%C3%A0-vendre-%C3%A0-bd-derfoufi-3-chambres-meubl%C3%A9


In [549]:

# price_difference



In [550]:
similare_att

,surface(m²),pieces,chambres,salles de bains,etat,etage,ans,ville,autre,prix(DHs),lienImage,lienArticle
7745,119 m²,8 Pièces,3 Chambres,2 Salles de bains,Bon état,2ème\n\t\t\t\t\tétage,1-5 ans,Bd Derfoufi à Oujda,Orientation: Nord Garage Ascenseur Concierg...,910000 DH,https://www.mubawab-media.com/ad/7/779/889F/h/...,https://www.mubawab.ma/fr/a/7779889/appartemen...


In [551]:
df['similarity'].max()

0.9999999999942908

In [552]:
((df==0).sum()/df.shape[0]).sort_values(ascending=False)[225:245]

1to5           0.72
porte          0.72
sécurité       0.69
salon          0.69
équipée        0.68
cuisine        0.68
terrasse       0.67
concierge      0.64
ascenseur      0.62
extérieure     0.60
façade         0.60
bonétat        0.53
1etage         0.50
nouveau        0.48
carrelage      0.42
5to10          0.36
prix(DHs)      0.00
surface(m²)    0.00
oujda          0.00
pieces         0.00
dtype: float64

In [553]:
df.columns[-1]

'similarity'

In [554]:
df[df['tanger']!=0]['tanger']

Series([], Name: tanger, dtype: float64)